In [149]:
import pymysql
import pandas as pd

# Database connection parameters
host = "localhost"
user = "slurm"
password = "slurm"
database = "slurmdb_micro3"

In [150]:
conn = pymysql.connect(host=host, user=user, passwd=password, db=database)
cursor = conn.cursor()

In [151]:
cursor.execute("SHOW TABLES")
tables = cursor.fetchall()
for table_name in tables:
    print(table_name)

('acct_coord_table',)
('acct_table',)
('clus_res_table',)
('cluster_table',)
('convert_version_table',)
('federation_table',)
('micro_assoc_table',)
('micro_assoc_usage_day_table',)
('micro_assoc_usage_hour_table',)
('micro_assoc_usage_month_table',)
('micro_event_table',)
('micro_job_table',)
('micro_last_ran_table',)
('micro_resv_table',)
('micro_step_table',)
('micro_suspend_table',)
('micro_usage_day_table',)
('micro_usage_hour_table',)
('micro_usage_month_table',)
('micro_wckey_table',)
('micro_wckey_usage_day_table',)
('micro_wckey_usage_hour_table',)
('micro_wckey_usage_month_table',)
('qos_table',)
('res_table',)
('table_defs_table',)
('tres_table',)
('txn_table',)
('user_table',)


In [152]:
cursor.close()
conn.close()

In [153]:
import pymysql
import pandas as pd

class SlurmDataBase:
    def __init__(self, host, user, password, database, cluster_prefix):
        self.host = host
        self.user = user
        self.password = password
        self.database = database
        self.cluster_prefix = cluster_prefix
    
    def read_table(self, query, query_params={}, df=True, verbose=False):
        conn = pymysql.connect(
            host=self.host,
            user=self.user,
            password=self.password,
            database=self.database
        )
        cursor = conn.cursor()
    
        final_query = query.format(**query_params)
        if verbose:
            print(final_query)
        cursor.execute(final_query)
        rows = cursor.fetchall()
        result = rows if not df else pd.DataFrame(rows, columns=[x[0] for x in cursor.description])
        
        cursor.close()
        conn.close()
        return result
    
    def read_running_tasks(self, columns=["*"], df=True, verbose=False):
        RUNNING_TASKS_QUERY = "select {columns} from {cluste_prefix}_job_table where time_start > 0 and time_end = 0"
        return self.read_table(RUNNING_TASKS_QUERY, {"columns": ', '.join(columns), "cluste_prefix": self.cluster_prefix}, df=df, verbose=verbose)

    def read_queued_tasks(self, columns=["*"], df=True, verbose=False):
        RUNNING_TASKS_QUERY = "select {columns} from {cluste_prefix}_job_table where time_submit > 0 and time_start = 0"
        return self.read_table(RUNNING_TASKS_QUERY, {"columns": ', '.join(columns), "cluste_prefix": self.cluster_prefix}, df=df, verbose=verbose)

In [154]:
database = SlurmDataBase(
    host = "localhost",
    user = "slurm",
    password = "slurm",
    database = "slurmdb_micro3",
    cluster_prefix = "micro"
)

In [155]:
table = database.read_table("select * from micro_job_table")

In [159]:
table[['time_start', 'time_end']]

,time_start,time_end
0,1712827213,1712827223
1,1712827240,1712827250
2,1712827240,1712827250
3,1712827240,1712827250
4,1712827240,1712827250
...,...,...
8811,1712850211,1712850221
8812,1712850221,1712850231
8813,1712850214,1712850224
8814,1712850211,1712850221
